<a href="https://colab.research.google.com/github/sue-bin/DA-session-quest/blob/master/1031_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification 과제
- San Francisco에서 일어난 범죄 관련 데이터 입니다. 설명변수들을 해석해서 범죄의 category를 분류하시면 됩니다~
- 캐글 competition 데이터 입니다. 혹시라도 FE에서 어려움을 겪으신다면 https://www.kaggle.com/c/sf-crime/overview 를 살짝 참고해보세용

In [38]:
import pandas as pd
import numpy as np
from collections import Counter

In [5]:
df = pd.read_csv('/content/drive/My Drive/YBIGTA/DA 교육세션/2020-10-31 ML  Classification/과제/sf_crime.csv')
df.head(10)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
5,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564
8,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601
9,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802


- Dates - 범죄 발생 timestamp
- Category - 범죄 사건의 카테고리 (이게 target variable 입니다!)
- Descript - 범죄 사건에 대한 자세한 설명
- DayOfWeek - 무슨 요일에 발생했는지
- PdDistrict - 관할 경찰서
- Resolution - 범죄 사건이 어떻게 처리되었는지
- Address - 범죄 발생이 일어난 위치와 가장 가까운 거리
- X - 경도
- Y - 위도

## 1. 데이터 형태 확인 및 전처리

In [ ]:
df.size()

In [8]:
df.dtypes

Dates          object
Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
dtype: object

In [11]:
df.isnull().sum()

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64

In [34]:
df.columns[[1,2,4,5]]

Index(['Category', 'Descript', 'PdDistrict', 'Resolution'], dtype='object')

In [43]:
for col in list(df.columns[[1,2,4,5]]):
  print("column: ",col)
  display(Counter(df[col]))

column:  Category


Counter({'ARSON': 1513,
         'ASSAULT': 76876,
         'BAD CHECKS': 406,
         'BRIBERY': 289,
         'BURGLARY': 36755,
         'DISORDERLY CONDUCT': 4320,
         'DRIVING UNDER THE INFLUENCE': 2268,
         'DRUG/NARCOTIC': 53971,
         'DRUNKENNESS': 4280,
         'EMBEZZLEMENT': 1166,
         'EXTORTION': 256,
         'FAMILY OFFENSES': 491,
         'FORGERY/COUNTERFEITING': 10609,
         'FRAUD': 16679,
         'GAMBLING': 146,
         'KIDNAPPING': 2341,
         'LARCENY/THEFT': 174900,
         'LIQUOR LAWS': 1903,
         'LOITERING': 1225,
         'MISSING PERSON': 25989,
         'NON-CRIMINAL': 92304,
         'OTHER OFFENSES': 126182,
         'PORNOGRAPHY/OBSCENE MAT': 22,
         'PROSTITUTION': 7484,
         'RECOVERED VEHICLE': 3138,
         'ROBBERY': 23000,
         'RUNAWAY': 1946,
         'SECONDARY CODES': 9985,
         'SEX OFFENSES FORCIBLE': 4388,
         'SEX OFFENSES NON FORCIBLE': 148,
         'STOLEN PROPERTY': 4540,
     

column:  Descript


Counter({'ABANDONMENT OF CHILD': 101,
         'ABORTION': 6,
         'ACCESS CARD INFORMATION, PUBLICATION OF': 8,
         'ACCESS CARD INFORMATION, THEFT OF': 311,
         'ACCIDENTAL BURNS': 2,
         'ACCIDENTAL SHOOTING': 13,
         'ACTS AGAINST PUBLIC TRANSIT': 325,
         'ADVERTISING DISTRIBUTORS PERMIT VIOLATION': 34,
         'AEROSOL CONTAINER; SALE, PURCHASE OR POSSESSION OF': 24,
         'AFFIXING ADVERTISMENTS TO POLES': 3,
         'AGGRAVATED ASSAULT OF POLICE OFFICER, SNIPING': 1,
         'AGGRAVATED ASSAULT OF POLICE OFFICER,BODILY FORCE': 170,
         'AGGRAVATED ASSAULT ON POLICE OFFICER WITH A GUN': 6,
         'AGGRAVATED ASSAULT ON POLICE OFFICER WITH A KNIFE': 19,
         'AGGRAVATED ASSAULT WITH A DEADLY WEAPON': 6451,
         'AGGRAVATED ASSAULT WITH A GUN': 1071,
         'AGGRAVATED ASSAULT WITH A KNIFE': 2442,
         'AGGRAVATED ASSAULT WITH BODILY FORCE': 4898,
         'AGGRESSIVE SOLICITING': 170,
         'AID OR HARBOR FELON': 25,
    

column:  PdDistrict


Counter({'BAYVIEW': 89431,
         'CENTRAL': 85460,
         'INGLESIDE': 78845,
         'MISSION': 119908,
         'NORTHERN': 105296,
         'PARK': 49313,
         'RICHMOND': 45209,
         'SOUTHERN': 157182,
         'TARAVAL': 65596,
         'TENDERLOIN': 81809})

column:  Resolution


Counter({'ARREST, BOOKED': 206403,
         'ARREST, CITED': 77004,
         'CLEARED-CONTACT JUVENILE FOR MORE INFO': 217,
         'COMPLAINANT REFUSES TO PROSECUTE': 3976,
         'DISTRICT ATTORNEY REFUSES TO PROSECUTE': 3934,
         'EXCEPTIONAL CLEARANCE': 1530,
         'JUVENILE ADMONISHED': 1455,
         'JUVENILE BOOKED': 5564,
         'JUVENILE CITED': 3332,
         'JUVENILE DIVERTED': 355,
         'LOCATED': 17101,
         'NONE': 526790,
         'NOT PROSECUTED': 3714,
         'PROSECUTED BY OUTSIDE AGENCY': 2504,
         'PROSECUTED FOR LESSER OFFENSE': 51,
         'PSYCHOPATHIC CASE': 14534,
         'UNFOUNDED': 9585})

In [ ]:
#Description 행의 값들이 너무 많아 FE 필요해보임


## 2. Feature Engineering 진행
- Insight 하나) 위도와 경도를 하나로 합쳐줄 수 있지 않을까요?

In [ ]:
df["X,Y"] = [()]*len(df)
for i in range(len(df)):
  df["X,Y"][i] = (df["X"][i],df["Y"][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
df

In [56]:
(df["X"][0],df["Y"][0])

(-122.425891675136, 37.7745985956747)

In [55]:
df

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,"X,Y"
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,-122.425892
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,-122.425892
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,-122.424363
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,-122.426995
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,-122.438738
...,...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,-122.459033
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948,-122.447364
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266,-122.403390
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607,-122.390531


## 3. 범주형 변수들 더미화

## 4. Train/Test Split

## 5. Decision Tree에 넣고 결과 확인, Feature Importance 까지 확인해보기

## 6. Random Forest, LGBM, XGBoost 중 하나에 넣고 결과 확인

어떤 알고리즘을 사용하셨나요? 왜 그 알고리즘을 사용하셨나요?